# Named entity recognition

In [1]:
import chaine

In [2]:
tokens = [["John", "Lennon", "was", "rhythm", "guitarist" "of", "The", "Beatles", "."]]
labels = [["B-PER", "I-PER", "O", "O", "O" "O", "B-ORG", "I-ORG", "O"]]

In [3]:
crf = chaine.train(tokens, labels, max_iterations=5)

[2020-12-06 23:50:12,783] [INFO] Loading data
[2020-12-06 23:50:12,785] [INFO] Start training
[2020-12-06 23:50:12,789] [INFO] Iteration: 1	Loss: 14.334076
[2020-12-06 23:50:12,792] [INFO] Iteration: 2	Loss: 14.334064
[2020-12-06 23:50:12,793] [INFO] Iteration: 3	Loss: 14.334053
[2020-12-06 23:50:12,794] [INFO] Iteration: 4	Loss: 14.334041
[2020-12-06 23:50:12,796] [INFO] Iteration: 5	Loss: 14.334029


In [4]:
crf.predict(tokens)

[['I-PER', 'I-PER', 'O', 'O', 'OO', 'B-ORG', 'I-ORG', 'O']]

## Feature extraction

```
identity of wi, identity of neighboring words
embeddings for wi, embeddings for neighboring words
part of speech of wi, part of speech of neighboring words
base-phrase syntactic chunk label of wi and neighboring words
presence of wi in a gazetteer
wi contains a particular prefix (from all prefixes of length ≤ 4)
wi contains a particular suffix (from all suffixes of length ≤ 4)
wi is all upper case
word shape of wi, word shape of neighboring words
short word shape of wi, short word shape of neighboring words
presence of hyphen
```